In [3]:
import pickle
import pandas as pd
regressor = pickle.load( open( "saved_model", "rb" ) )
regressor

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=nan, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

array([0.12287334, 0.0831758 , 0.04914934, 0.03402647, 0.02835539,
       0.03213611, 0.0926276 , 0.02646503, 0.03780719, 0.0926276 ,
       0.01890359, 0.0415879 , 0.01701323, 0.03213611, 0.        ,
       0.01134215, 0.02457467, 0.01890359, 0.        , 0.00567108,
       0.01134215, 0.01701323, 0.10964083, 0.03591682, 0.00378072,
       0.05293006], dtype=float32)

In [5]:
grid_processed = pd.read_csv("./data/grid_processed.csv", index_col=0)

In [6]:
grid_processed['budynki_niemieszkalne_9'] = grid_processed['budynki_all_9'] - grid_processed['budynki_mieszkalne_9']
grid_processed['populacja_na_budynek_9'] = grid_processed['populacja_razem_9'] - grid_processed['budynki_mieszkalne_9']
grid_processed['budynki_niemieszkalne_na_populacje_9'] = grid_processed['budynki_niemieszkalne_9'] / (grid_processed['populacja_razem_9'] + 1)
X_grid = grid_processed.drop(['longitude', 'latitude'], axis=1)

In [11]:
sorted([tup for tup in zip(X_grid.columns, regressor.feature_importances_)], key=lambda x: x[1])

[('Oddział Banku_1000', 0.0),
 ('Hipermarket_1000', 0.0),
 ('populacja_na_budynek_9', 0.0037807184),
 ('Hipermarket_2000', 0.0056710774),
 ('Oddział Banku_2000', 0.011342155),
 ('Stacja Paliw_1000', 0.011342155),
 ('Przystanek autobusowy_1000', 0.017013233),
 ('Stacja Paliw_3000', 0.017013233),
 ('budynki_mieszkalne_16', 0.018903593),
 ('Przystanek tramwajowy_2000', 0.018903593),
 ('Przystanek tramwajowy_1000', 0.02457467),
 ('budynki_all_16', 0.026465029),
 ('populacja_razem_16', 0.028355388),
 ('populacja_razem_25', 0.032136105),
 ('Przystanek autobusowy_2000', 0.032136105),
 ('populacja_razem_9', 0.034026466),
 ('budynki_niemieszkalne_9', 0.035916824),
 ('budynki_all_25', 0.037807185),
 ('budynki_mieszkalne_25', 0.0415879),
 ('dochod_bud_pra_25', 0.04914934),
 ('budynki_niemieszkalne_na_populacje_9', 0.052930057),
 ('dochod_bud_pra_16', 0.0831758),
 ('budynki_all_9', 0.0926276),
 ('budynki_mieszkalne_9', 0.0926276),
 ('elevation', 0.10964083),
 ('dochod_bud_pra_9', 0.12287334)]

In [ ]:
grid_out = regressor.predict(X_grid)